In [29]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install biopython

In [30]:
from Bio import SeqIO

#Ex.1: read file, one hot encode, decode; use an example

#Step 1: read file - sequence + functionality
#Create dictionary: keys = sequence_n; values = prot sequence, functionality
file = "./MSA_nat_with_annotation.faa"
data_dict = {}

for seq_record in SeqIO.parse(file, "fasta"):
    if seq_record.description.replace(seq_record.id+' ','',1) == "functional_true":
        label = 1
    else:
        label = 0
    data_dict[seq_record.id] = seq_record.seq, label
    #print(seq_record.id)
    #print(seq_record.description.replace(seq_record.id+' ','',1))
    #print(repr(seq_record.seq))
    #print(len(seq_record))

print(data_dict.get("sequence_1")[0])
print(data_dict.get("sequence_1")[1])
print(data_dict.get("sequence_2")[0])
print(data_dict.get("sequence_2")[1])

-TSENPLLALREKISALDEKLLALLAERRELAVEVGKAKLLSHRPVRDIDRERDLLERLITLGK-AHHLDAHYITRLFQLIIEDSVLTQQALLQQH
1
---DERIQALRKEVDRVNREILRLLSERGRLVQEIGRLQTELGLPHYDPKREEEMLAYLTAEN--PGPFPDETIRKLFKEIFKASLDLEERQDQ--
0


In [81]:
#One hot encoding
import numpy as np
from numpy import argmax, shape

# define alphabet of possible input values
alphabet ='ACDEFGHIKLMNPQRSTVWY-'

def one_hot_encoder(MSA_str, alphabet):
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in MSA_str]
    # one hot encode
    onehot_encoded = list()
    for value in integer_encoded:
        if value == char_to_int.get("-"):
            letter = [0 for value in range(len(alphabet)-1)]
        else:
            letter = [0 for value in range(len(alphabet)-1)]
            letter[value] = 1
        onehot_encoded.append(letter)
    return onehot_encoded

#cuz you don't set a lock without getting a key
def one_hot_decoder(encoded_str, alphabet):
    # define a mapping of integers to chars
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    decoded = ''
    for i in range(shape(encoded_str)[0]):
        if np.count_nonzero(encoded_str[i]) == 0:
            inverted = '-'
        else:
            inverted = int_to_char[argmax(encoded_str[i])]
        decoded += inverted
    return decoded

#Example
tmp_sequence = data_dict.get("sequence_1")[0]
encoded = one_hot_encoder(tmp_sequence, alphabet)
print(shape(encoded))
decoded = one_hot_decoder(encoded, alphabet)
print(tmp_sequence)
print(decoded)

tmp_sequence = "---"
encoded = one_hot_encoder(tmp_sequence, alphabet)
print(encoded)
decoded = one_hot_decoder(encoded, alphabet)
print(decoded)

(96, 20)
-TSENPLLALREKISALDEKLLALLAERRELAVEVGKAKLLSHRPVRDIDRERDLLERLITLGK-AHHLDAHYITRLFQLIIEDSVLTQQALLQQH
-TSENPLLALREKISALDEKLLALLAERRELAVEVGKAKLLSHRPVRDIDRERDLLERLITLGK-AHHLDAHYITRLFQLIIEDSVLTQQALLQQH
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
---
